# Install

### References
* Official
  * wiki
    * https://github.com/BVLC/caffe/wiki
  * Homepage
    * http://caffe.berkeleyvision.org/
* Others 
  * https://hpc.uiowa.edu/sites/hpc.uiowa.edu/files/wysiwyg_uploads/ModifiedCaffeTutorial_0.pdf

### Prerequirements
* DB (http://caffe.berkeleyvision.org/tutorial/data.html)
  * level db
    * fast key-value storage library written at Google
    * https://github.com/google/leveldb
  * lmdb
    * lightning memory-mapped database
    * https://en.wikipedia.org/wiki/Lightning_Memory-Mapped_Database
* protobuf
  * for serialize/deserialize
  * https://en.wikipedia.org/wiki/Protocol_Buffers
* hdf5
  * file format, Hierarchical Data Format
  * https://en.wikipedia.org/wiki/Hierarchical_Data_Format
* glog
  * logging library from google
  * https://github.com/google/glog
* gflags
  * command line flag parser
  * https://github.com/gflags/gflags
  
  
* for python
  * for pygraphviz, which pydot is dependent on
    * PyGraphviz doesn't work without Graphviz
    * `sudo apt-get install graphviz`
  * `pip3 install pydot scikit-image pytest`
  
```sh
sudo apt-get install libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler
sudo apt-get install --no-install-recommends libboost-all-dev
sudo apt-get install libatlas-base-dev
sudo apt-get install libgflags-dev libgoogle-glog-dev liblmdb-dev
```


### Fix

```diff
diff --git a/Makefile b/Makefile
index 4d32416..ad173f8 100644
--- a/Makefile
+++ b/Makefile
@@ -178,7 +178,7 @@ ifneq ($(CPU_ONLY), 1)
    LIBRARIES := cudart cublas curand
 endif
 
-LIBRARIES += glog gflags protobuf boost_system boost_filesystem m hdf5_hl hdf5
+LIBRARIES += glog gflags protobuf boost_system boost_filesystem m hdf5_serial_hl hdf5_serial
 
 # handle IO dependencies
 USE_LEVELDB ?= 1
```

### Configurations

#### Makefile.config
* for Multi-GPU
```
USE_NCCL := 1
```
  * install nccl (http://www.nvidia.com/object/caffe-installation.html)
```
$ git clone https://github.com/NVIDIA/nccl.git
$ cd nccl
$ sudo make install -j4
```

* for cudnn
```
USE_CUDNN:=1
```
* for opencv
```
OPENCV_VERSION:=3
```
* for python3
```
PYTHON_LIBRARIES := boost_python-py35 python3.5m
PYTHON_INCLUDE := /usr/include/python3.5m \
                 /usr/lib/python3.5/dist-packages/numpy/core/include
```
* Custumize include and library dir
  * e.g. I installed OpenCV with `prefix=/opt/` so I need `/opt/include`, `/opt/lib`
```
INCLUDE_DIRS := $(PYTHON_INCLUDE) /usr/local/include /usr/include/hdf5/serial/ /opt/include/
LIBRARY_DIRS := $(PYTHON_LIB) /usr/local/lib /usr/lib /opt/lib
```

#### Makefile

* for python3

```diff
@@ -533,7 +533,7 @@ runtest: $(TEST_ALL_BIN)
    $(TEST_ALL_BIN) $(TEST_GPUID) --gtest_shuffle $(TEST_FILTER)
 
 pytest: py
-   cd python; python -m unittest discover -s caffe/test
+   cd python; python3 -m unittest discover -s caffe/test
 
 mattest: mat
    cd matlab; $(MATLAB_DIR)/bin/matlab -nodisplay -r 'caffe.run_tests(), exit()'
```

### How to run my own code
* ```export PYTHONPATH=~/work/caffe/ssd/python/```
* Easy way
  * put a **single** `*.cpp` file in ./examples
* Build own project by linking caffe.so???
  * 아직 모른다.

### save/load caffemodel

* save
  * declared in solver.prototxt
```
# snapshot intermediate results
snapshot: 5000
snapshot_prefix: "examples/mnist/lenet"
```
  * you can see `*.caffemodel`, `*.solverstate`
```
-rw-rw-r--  1 rofox rofox 1725006 May 25 10:59 lenet_iter_10000.caffemodel
-rw-rw-r--  1 rofox rofox 1724471 May 25 10:59 lenet_iter_10000.solverstate
-rw-rw-r--  1 rofox rofox 1725006 May 25 10:58 lenet_iter_5000.caffemodel
-rw-rw-r--  1 rofox rofox 1724470 May 25 10:58 lenet_iter_5000.solverstate
```

* load
  * 

### Multi GPU Test

* cifar10 test total batch 100
  
| 1 gpu | 2 gpu | 4 gpu
:-:|:-:|:-:|:-:
local_batch x num_gpus| 100x1 | 50x2 | 25x4
performance (iters/sec)| 30.2061 | 49.4755 | 57.3761
ratio| 1 | 1.6379 | 1.8994